# Global averaging

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.signal import detrend
from matplotlib import pyplot as plt
from scipy import signal
import pandas as pd
import xarray as xr
import intake
import pprint
import util 
import os

col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

file = 'available_tas_data_cloud_june12th_2020.txt'

In [2]:
# load table:
data_table = pd.read_table(file,index_col=0)
models_used = data_table['piControl (yrs)'].index
print(models_used)

Index(['IPSL-CM6A-LR', 'MRI-ESM2-0', 'MPI-ESM1-2-LR', 'TaiESM1',
       'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1',
       'CAMS-CSM1-0', 'FGOALS-f3-L', 'FGOALS-g3', 'IITM-ESM', 'CanESM5',
       'CanESM5-CanOE', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1',
       'ACCESS-ESM1-5', 'ACCESS-CM2', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA',
       'EC-Earth3', 'EC-Earth3-LR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR',
       'FIO-ESM-2-0', 'MPI-ESM-1-2-HAM', 'INM-CM4-8', 'INM-CM5-0',
       'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
       'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'GISS-E2-1-G', 'GISS-E2-1-G-CC',
       'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM',
       'CESM2-WACCM-FV2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM',
       'KACE-1-0-G', 'GFDL-CM4', 'GFDL-ESM4', 'NESM3', 'SAM0-UNICON', 'CIESM',
       'MCM-UA-1-0'],
      dtype='object')


In [3]:
# load functions:
def area_weights(lat_bnds, lon_bnds): 
    # computes exact area weigths assuming earth is a perfect sphere
    lowerlats = np.radians(lat_bnds[:,0]); upperlats = np.radians(lat_bnds[:,1])
    difflon = np.radians(np.diff(lon_bnds[0,:])) # if the differences in longitudes are all the same
    areaweights = difflon*(np.sin(upperlats) - np.sin(lowerlats));
    areaweights /= areaweights.mean()
    return areaweights # list of weights, of same dimension as latitude

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def leap_year(year, calendar='standard'):
    """Determine if year is a leap year"""
    leap = False
    if ((calendar in ['standard', 'gregorian',
        'proleptic_gregorian', 'julian']) and
        (year % 4 == 0)):
        leap = True
        if ((calendar == 'proleptic_gregorian') and
            (year % 100 == 0) and
            (year % 400 != 0)):
            leap = False
        elif ((calendar in ['standard', 'gregorian']) and
                 (year % 100 == 0) and (year % 400 != 0) and
                 (year < 1583)):
            leap = False
    return leap

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
        if leap_year(year, calendar=calendar) and month == 2:
            month_length[i] += 1
    return month_length

# days per month:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'julian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], ##### I think this should be correct
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
      }

def compute_day_weights(ds, calendar = 'noleap', first_month = 1): # new function
    month_length = xr.DataArray(get_dpm((ds.time.to_index()), calendar=ds_calendar), coords=[ds.time], name='month_length')
    
    ##### This code is only tested for noleap so far #####
    if first_month == 1:
        norm_by_annual = month_length.groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
    else: 
        norm_by_annual = np.array([month_length[i*12:(i+1)*12].mean() for i in range(int(len(month_length)/12))])
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual[i], 12) for i in range(len(norm_by_annual))])
        
    weights = month_length/norm_by_monthly
    # normalized to have mean 1
    return weights 

def calendar_check(model):
    # Time formats for piControl, found from manual check:
    if model in ['TaiESM1', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'FGOALS-f3-L', 'FGOALS-g3', 'CanESM5', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'FIO-ESM-2-0', 'INM-CM4-8', 'INM-CM5-0', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM', 'GFDL-CM4', 'SAM0-UNICON', 'GFDL-ESM4', 'CIESM', 'MCM-UA-1-0']:
        ds_calendar = 'noleap'
    elif model in ['EC-Earth3', 'CNRM-CM6-1', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'NESM3']: # 'IPSL-CM6A-LR':'piClim-4xCO2','piClim-control' says noleap calendar
        ds_calendar = 'gregorian'
    elif model in ['AWI-CM-1-1-MR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'ACCESS-ESM1-5', 'ACCESS-CM2', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-LR', 'MPI-ESM1-2-HR', 'EC-Earth3-LR']:
        ds_calendar = 'proleptic_gregorian'
    elif model in ['UKESM1-0-LL', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'CNRM-ESM2-1', 'KACE-1-0-G', 'MRI-ESM2-0']:
        ds_calendar = '360_day'
        if model in ['CNRM-ESM2-1', 'MRI-ESM2-0']:
            print('piControl is 360_day, the other experiments unknown')
    elif model in ['IITM-ESM']:
        ds_calendar = 'julian'
    elif model in ['AWI-ESM-1-1-LR', 'CNRM-CM6-1-HR', 'EC-Earth3', 'EC-Earth3-LR']:
        #ds_calendar = 'datetime64'
        print('not 100% sure what calendar this model has, but a guess is made based on other models from same institution')
        if model in ['AWI-ESM-1-1-LR']:
            print('calendar is likely proleptic gregorian')
            ds_calendar = 'proleptic_gregorian'
        elif model in ['CNRM-CM6-1-HR']:
            print('calendar is likely gregorian')
            ds_calendar = 'gregorian'
    return ds_calendar

# calendar double checked for models:
# ACCESS-CM2: proleptic_gregorian

# 'IPSL-CM6A-LR', 'abrupt-4xCO2': gregorian
# 'EC-Earth3-LR', 'piControl': proleptic_gregorian

    

# Choose model, exp, member

In [4]:
#model = models_used[10];
#model = 'CNRM-CM6-1'
#model = 'EC-Earth3' # should have spatial dimensions 256, 512
model = 'GISS-E2-1-G'
#model = 'ACCESS-CM2'
#model = 'HadGEM3-GC31-LL'
#model = 'MRI-ESM2-0'
#model = 'NorESM2-LM'

print(model)
ds_calendar = calendar_check(model)
print(ds_calendar, 'calendar')

#check what experiments are available for var
var = 'tas'
#cat = col.search(experiment_id = ['piControl', 'abrupt-4xCO2', 'abrupt-2xCO2', 'abrupt-0p5xCO2', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585'], source_id = model, variable_id=var, table_id='Amon') 
#cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
cat = col.search(experiment_id = 'ssp585', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = 'abrupt-4xCO2', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = 'piClim-control', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 


#cat.df
pd.set_option('display.max_colwidth', -1)
cat.df['zstore']

GISS-E2-1-G
noleap calendar


0     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p5f1/Amon/rsut/gn/v20200115/
1     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p5f1/Amon/tas/gn/v20200115/ 
2     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r2i1p1f2/Amon/tas/gn/v20200115/ 
3     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p5f1/Amon/rlut/gn/v20200115/
4     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p5f1/Amon/rsdt/gn/v20200115/
5     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r4i1p1f2/Amon/tas/gn/v20200115/ 
6     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r4i1p3f1/Amon/tas/gn/v20200115/ 
7     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p3f1/Amon/rlut/gn/v20200115/
8     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p3f1/Amon/rsdt/gn/v20200115/
9     gs://cmip6/CMIP6/ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp585/r1i1p3f1/Amon/rsut/gn/v20200115/
10    gs://cmip6/CMIP6/Scenari

In [34]:
#exp = 'piControl'
#exp = 'abrupt-4xCO2'
#exp = 'abrupt-2xCO2'
#exp = 'abrupt-0p5xCO2'
#exp = 'historical'
#exp = 'ssp126'
#exp = 'ssp245'
#exp = 'ssp370'
#exp = 'ssp585'
#exp = 'piClim-4xCO2'
#exp = 'piClim-control'
#exp = 'piClim-histall'


if model == 'UKESM1-0-LL':
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'MOHC')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'NIMS-KMA')
if model == 'MPI-ESM1-2-HR': # select institution if there are two groups
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DKRZ') 
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DWD') 
elif model in ['MPI-ESM1-2-LR', 'IPSL-CM6A-LR', 'MRI-ESM2-0', 'EC-Earth3'] and exp in ['piControl']: # problem when loading more than one member simultaneously, so specify member_id:
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', member_id = 'r2i1p1f1')
else:
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', member_id = 'r1i1p1f1')
    #                 member_id = ['r2i1p1f1','r3i1p1f1','r4i1p1f1','r5i1p1f1','r6i1p1f1','r7i1p1f1','r8i1p1f1','r9i1p1f1','r10i1p1f1','r11ip1f1','r12i1p1f1']) 
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['rlut'], table_id='Amon', member_id = 'r1i1p1f1')

# members with starting months different from jan should be loaded separately to ensure we get the correct time axis


if 'dset_dict' in locals():
    del dset_dict
if 'ds_exp' in locals():
    del ds_exp
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
#for key in ['CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn']:
for key in dset_dict.keys():
    ds_exp = dset_dict[key]
    if 'members_sorted' in locals():
        del members_sorted
    members_sorted = ds_exp.member_id.sortby(ds_exp.member_id)
    
if model == 'MCM-UA-1-0':
    ds_exp = ds_exp.rename({'longitude': 'lon','latitude': 'lat'}) 
        
for member in members_sorted:
    print(member.values)
    
# write out data variables, to check that we have all we want
ds_exp.data_vars


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


r1i1p1f1


Data variables:
    rlut     (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 114, 256, 512), meta=np.ndarray>
    rsdt     (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 148, 256, 512), meta=np.ndarray>
    rsut     (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 117, 256, 512), meta=np.ndarray>
    tas      (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 126, 256, 512), meta=np.ndarray>

In [35]:
ds_exp

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 256, lon: 512, member_id: 1, time: 1032)
Coordinates:
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(256, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(512, 2), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1032, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * time       (time) datetime64[ns] 2015-01-16T12:00:00 ... 2100-12-16T12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * member_id  (member_id) <U8 'r1i1p1f1'
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    rlut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 114, 256, 512), meta=np.ndarray>
    rsdt       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 148, 256, 512), meta=np.ndarray>
    rsut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 117, 256, 512), meta=np.ndarray>
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 126, 256, 512), meta=np.ndarray>
Attributes:
    sub_experiment:          none
    physics_index:           1
    source_type:             AOGCM
    frequency:               mon
    experiment:              update of RCP8.5 based on SSP5
    external_variables:      areacella
    source_id:               EC-Earth3
    Conventions:             CF-1.7 CMIP-6.2
    parent_mip_era:          CMIP6
    license:                 CMIP6 model data produced by EC-Earth-Consortium...
    realization_index:       1
    comment:                 Production: Thomas Reerink at KNMI
    branch_time:             0.0
    nominal_resolution:      100 km
    contact:                 cmip6-data@ec-earth.org
    parent_time_units:       days since 1850-01-01
    sub_experiment_id:       none
    parent_activity_id:      CMIP
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.EC-Earth-Co...
    activity_id:             ScenarioMIP
    mip_era:                 CMIP6
    version_id:              v20200310
    grid_label:              gr
    initialization_index:    1
    product:                 model-output
    realm:                   atmos
    title:                   EC-Earth3 output prepared for CMIP6
    forcing_index:           1
    experiment_id:           ssp585
    tracking_id:             hdl:21.14100/99ca9cae-9f45-41e1-9e0f-12060ae7450...
    cmor_version:            3.5.0
    grid:                    T255L91
    source:                  EC-Earth3 (2019): \naerosol: none\natmos: IFS cy...
    table_info:              Creation Date:(24 July 2019) MD5:70649eeb16bc90c...
    institution_id:          EC-Earth-Consortium
    branch_method:           standard
    variant_label:           r1i1p1f1
    table_id:                Amon
    data_specs_version:      01.00.31
    history:                 2019-09-27T17:10:13Z ; CMOR rewrote data to be c...
    parent_source_id:        EC-Earth3
    parent_variant_label:    r1i1p1f1
    parent_experiment_id:    historical
    branch_time_in_parent:   60265.0D
    branch_time_in_child:    60265.0D
    institution:             AEMET, Spain; BSC, Spain; CNR-ISAC, Italy; DMI, ...
    intake_esm_dataset_key:  ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585...

In [36]:
#ds_exp.time.to_index()
ds_exp.time.encoding['calendar']

'proleptic_gregorian'

In [37]:
if model in ['CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']: # French models missing lat_bnds and lon_bnds coordinates
    # if computing area-weights from areacella instead:
    #area_cat= col.search(source_id = model, variable_id=['areacella'])
    area_cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['areacella'])
    area_dset_dict = area_cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})

    for key in area_dset_dict.keys():
        area_ds = area_dset_dict[key]
    areas = area_ds['areacella'].values[0,:,0]
    norm_areas = areas/areas.mean()

# loop through members
#for member in [members_sorted.sel(member_id = 'r1i1p2f1')]:
for member in members_sorted:
    print(member.values)

    ds = ds_exp.sel(member_id = member)
    
    
    ds_calendar = ds.time.encoding['calendar']
    #ds_calendar = 'proleptic_gregorian'
    unit = ds.time.encoding['units']
    firstmonth = ds_exp.time.to_index().month[0] # not jan for IPSL-CM6A-LR members 2-12
    # annual average from feb-jan, march-feb, etc. when firstmonth is not jan. 
    
    # anomalies: does it matter if the piControl annual averages we subtract are averaged over jan-dec or other months? Probably not
    if model == 'MRI-ESM2-0' and exp == 'abrupt-4xCO2':
        # note: piControl values in these time series before the branch time
        rstring = str(member.values)[1:3].replace('i','')
        firstmonth = int(rstring)
        if firstmonth>1: # delete values before 'firstmonth', and some values in the end:
            #ds = ds.drop_isel(time=np.arange(firstmonth-1))
            ds = ds.isel(time=np.arange(firstmonth-1,len(ds.time)-(12-firstmonth+1)))
            # check result
            print(ds.time.to_index().month[0])
    elif model == 'CNRM-CM6-1' and exp == 'abrupt-4xCO2':
        # delete values in the end to get a multiple of 12 length for the annual mean computations
        ds = ds.isel(time=np.arange(len(ds.time)-(12-firstmonth+1)))
        
    
    print(ds_calendar, 'calendar')
    print(unit)
    print('first month of dataset is:', firstmonth)
    
    # compute weights for average
    if model == 'NorCPM1' and exp == 'historical':
        area_w = area_weights(ds.lat_bnds.values[0,:,:], ds.lon_bnds.values)
    elif model in ['CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']: # French models missing lat_bnds and lon_bnds coordinates
        area_w = norm_areas
        #print('maximum difference in area-weights is:', np.max(np.abs(norm_areas - area_w)))
        
        # differences in global means are noticeable from the third decimal for the low-res model BCC-ESM1
        # for this model maximum difference in area-weights is: 0.003299741144367374
        
        # differences in global means are noticeable from the 5th decimal for the model CESM2
        # for this model maximum difference in area-weights is: 1.2547306571519812e-07
    else:
        area_w = area_weights(ds.lat_bnds.values, ds.lon_bnds.values)
        
    day_weights = compute_day_weights(ds, calendar = ds_calendar, first_month = firstmonth)

    varlist = ['tas', 'rlut', 'rsut', 'rsdt']
    
    for variable in varlist:
        print(variable)
        data = ds[variable]
                
        # global average
        area_avg = (data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])

        # annual average
        day_weighted_avg = area_avg*day_weights
        if firstmonth == 1 or firstmonth == 13:
            annualmean = day_weighted_avg.groupby('time.year').mean('time')
        else: # this code is slow, but rarely needed
            annualmean_array = np.array([day_weighted_avg[i*12:(i+1)*12].mean() for i in range(int(len(day_weighted_avg)/12))])
            annualmean = xr.DataArray(annualmean_array)
            
        if variable == varlist[0]:
            # create dataframe for storing all results
            df = pd.DataFrame(annualmean.values, columns = [variable])
        else:
            df_col = pd.DataFrame(annualmean.values, columns = [variable])
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
            
    filename = model + '_' + exp + '_' + str(member.values) + '_means.txt'
    file = os.path.join('../Processed_data/Global_annual_means/', model, exp, filename)
    if member == members_sorted[0]: # create directory for first member
        os.makedirs(os.path.dirname(file), exist_ok=False)

    df.to_csv(file)
    

r1i1p1f1
proleptic_gregorian calendar
hours since 2015-01-16 12:00:00.000000
first month of dataset is: 1
tas
rlut
rsut
rsdt


In [38]:
ds.time


<xarray.DataArray 'time' (time: 1032)>
array(['2015-01-16T12:00:00.000000000', '2015-02-15T00:00:00.000000000',
       '2015-03-16T12:00:00.000000000', ..., '2100-10-16T12:00:00.000000000',
       '2100-11-16T00:00:00.000000000', '2100-12-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time       (time) datetime64[ns] 2015-01-16T12:00:00 ... 2100-12-16T12:00:00
    member_id  <U8 'r1i1p1f1'
    height     float64 ...
Attributes:
    axis:           T
    bounds:         time_bnds
    long_name:      time
    standard_name:  time

In [39]:
df

,tas,rlut,rsut,rsdt
0,288.424323,241.671600,97.919629,340.325112
1,288.391751,241.544819,98.271372,340.284045
2,288.337440,241.232012,98.190634,340.233501
3,288.458722,241.482928,97.973053,340.225824
4,288.480679,241.477527,98.422058,340.224582
...,...,...,...,...
81,292.665783,243.479728,93.974720,340.263946
82,292.859553,243.823489,93.290931,340.218986
83,292.958576,243.882666,94.175667,340.205871
84,293.307612,244.371311,93.795841,340.224508


In [ ]:
ds

In [ ]:
514/2

In [ ]:
#area_ds
df

In [ ]:
#area_w_df = df
#area_w_df

In [ ]:
#area_w_df - df

In [ ]:
# NorCPM1 historical: nans in lat_bnds

# Updated: NorESM, NorCPM, CESM2, GISS


In [ ]:
model

In [ ]:
# check dataset availability:
#col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
#cat = col.search(experiment_id = 'piClim-4xCO2', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'piClim-control', source_id = 'BCC-ESM1', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'piClim-control', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'abrupt-0p5xCO2', source_id = 'CESM2', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
cat = col.search(experiment_id = 'piControl', source_id = 'CNRM-CM6-1', variable_id=['areacella'])
cat.df

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})

In [ ]:
# latitude boundaries are not just simple means of latitude coordinates..
# reconstruct lon_bnds for French models missing this coordinate
#lon_bnds_1dim = (ds_exp.lon[1:].values + ds_exp.lon[0:-1].values)/2
#lon_bnds_2dim = np.array([[lon_bnds_1dim[i], lon_bnds_1dim[i+1]] for i in range(len(ds_exp.lon)-2)]) # middle values
#lon_bnds_2dim = np.append([[-lon_bnds_2dim[0,0], lon_bnds_2dim[0,0]]],lon_bnds_2dim, axis = 0) # add first values
#lon_bnds_2dim = np.append(lon_bnds_2dim, [[lon_bnds_2dim[-1,1], (360+ds_exp.lon[0]+ds_exp.lon[-1])/2]], axis = 0) # add last values

#ds_exp['tas'][0,0,0,0:10].values
#ds_exp['rlut'][0,0,0,0:10].values

#for member in members_sorted:
#    print(member.values)
#
#    ds = ds_exp.sel(member_id = member)
#    print(ds_exp.sizes)
#    print(ds.lon_bnds[0:10].values)
#    print(ds.lat_bnds[0:10].values)

#ds_exp.grid

#ds_exp.rlut.isel(time = 10).plot()
#dset_dict.keys()

# testing model EC earth3
#ds_test = ds_exp.sel(member_id = 'r4i1p1f1')
#ds_test['tas'].isel(time=0).values[0]
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#ds_test['tas'].isel(time=0).values[0].shape
#ds_test.lon

#differences = norm_areas-area_w
#differences
#plt.plot(differences[1:-1])
#col.search(source_id = model, variable_id=['areacella']).df